Load and tokenize data

In [1]:
from __future__ import print_function

from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np

batch_size = 64  # Batch size for training.
epochs = 50  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 1000  # Number of samples to train on.
# Path to the data
data_path = 'datasets/enfratexts.txt'

# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')
for line in lines[: min(num_samples, len(lines) - 1)]:
    target_text, input_text, _ = line.split('\t')
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('Number of samples:', len(input_texts))
print('Number of input tokens:', num_encoder_tokens)
print('Number of output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)


Using TensorFlow backend.


Number of samples: 1000
Number of unique input tokens: 75
Number of unique output tokens: 65
Max sequence length for inputs: 31
Max sequence length for outputs: 12


Prepare input and output data

In [2]:
input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')


Prepare the encoder-decoder data by looping through all the texts

In [3]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    encoder_input_data[i, t + 1:, input_token_index[' ']] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.
    decoder_input_data[i, t + 1:, target_token_index[' ']] = 1.
    decoder_target_data[i, t:, target_token_index[' ']] = 1.


In [4]:
len(input_texts)

1000

Define the Encoder-Decoder Architecture

In [5]:
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)


Run a few epochs (50 only, and on 1000 documents, to save memory and time)

In [6]:
# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)
# Save model
model.save('encode_decode_model')

Train on 800 samples, validate on 200 samples
Epoch 1/50
800/800 [==============================] - 5s 7ms/step - loss: 3.0706 - accuracy: 0.2444 - val_loss: 2.9336 - val_accuracy: 0.1800
Epoch 2/50
800/800 [==============================] - 5s 6ms/step - loss: 2.5766 - accuracy: 0.2976 - val_loss: 2.7795 - val_accuracy: 0.1800
Epoch 3/50
800/800 [==============================] - 4s 5ms/step - loss: 2.3889 - accuracy: 0.3365 - val_loss: 2.6962 - val_accuracy: 0.1733
Epoch 4/50
800/800 [==============================] - 4s 5ms/step - loss: 2.3302 - accuracy: 0.3529 - val_loss: 2.7575 - val_accuracy: 0.1946
Epoch 5/50
800/800 [==============================] - 4s 5ms/step - loss: 2.3043 - accuracy: 0.3724 - val_loss: 2.5277 - val_accuracy: 0.3358
Epoch 6/50
800/800 [==============================] - 5s 6ms/step - loss: 2.2236 - accuracy: 0.3833 - val_loss: 3.0984 - val_accuracy: 0.1688
Epoch 7/50
800/800 [==============================] - 4s 5ms/step - loss: 2.1859 - accuracy: 0.4132 - 

Inference

1) encode input and retrieve initial decoder state
2) run one step of decoder with this initial state and a "start of sequence" token as target. Output will be the next target token
3) Repeat with the current target token and current states

In [7]:

# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

# Reverse-lookup token index to decode sequences back to something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())




def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

Decoding test

In [8]:

for seq_index in range(100):
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

-
Input sentence: Va !
Decoded sentence: How nick!

-
Input sentence: Salut !
Decoded sentence: How nick!

-
Input sentence: Salut.
Decoded sentence: How nick!

-
Input sentence: Cours !
Decoded sentence: How nic!

-
Input sentence: Courez !
Decoded sentence: We this.

-
Input sentence: Qui ?
Decoded sentence: How nick!

-
Input sentence: Ça alors !
Decoded sentence: How ne!

-
Input sentence: Au feu !
Decoded sentence: How nic!

-
Input sentence: À l'aide !
Decoded sentence: Wat hom.

-
Input sentence: Saute.
Decoded sentence: How nick!

-
Input sentence: Ça suffit !
Decoded sentence: We lost.

-
Input sentence: Stop !
Decoded sentence: How nic!

-
Input sentence: Arrête-toi !
Decoded sentence: How nick!

-
Input sentence: Attends !
Decoded sentence: It's hed.

-
Input sentence: Attendez !
Decoded sentence: How nick!

-
Input sentence: Poursuis.
Decoded sentence: How nick!

-
Input sentence: Continuez.
Decoded sentence: How nick!

-
Input sentence: Poursuivez.
Decoded sentence: How ni